In [1]:
import joblib
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("algebra_2008_2009_train_trimmed3_remove_duplicates_sorted.csv", delimiter='\t')

In [3]:
id_to_cfar = {}
for stu_id, cfa in zip(data['Anon Student Id'], data['Correct First Attempt']):
    if stu_id not in id_to_cfar:
        id_to_cfar[stu_id] = [cfa, 1.]
    else:
        id_to_cfar[stu_id][0] += cfa
        id_to_cfar[stu_id][1] += 1.

for key, value in id_to_cfar.iteritems():
    id_to_cfar[key] = id_to_cfar[key][0] / id_to_cfar[key][1]
    


In [4]:
unit_to_cfar = {}

for unit, cfa in zip(data['Unit'], data['Correct First Attempt']):
    if unit not in unit_to_cfar:
        unit_to_cfar[unit] = [cfa, 1.]
    else:
        unit_to_cfar[unit][0] += cfa
        unit_to_cfar[unit][1] += 1.
        
for key, value in unit_to_cfar.iteritems():
    unit_to_cfar[key] = unit_to_cfar[key][0] / unit_to_cfar[key][1]

#unit_to_cfar

In [5]:
section_to_cfar = {}

for section, cfa in zip(data['Sections'], data['Correct First Attempt']):
    if section not in section_to_cfar:
        section_to_cfar[section] = [cfa, 1.]
    else:
        section_to_cfar[section][0] += cfa
        section_to_cfar[section][1] += 1.
        
for key, value in section_to_cfar.iteritems():
    section_to_cfar[key] = section_to_cfar[key][0] / section_to_cfar[key][1]
    
#section_to_cfar

In [6]:
prob_to_cfar = {}

for prob, cfa in zip(data['Problem Name'], data['Correct First Attempt']):
    if prob not in prob_to_cfar:
        prob_to_cfar[prob] = [cfa, 1.]
    else:
        prob_to_cfar[prob][0] += cfa
        prob_to_cfar[prob][1] += 1.
        
for key, value in prob_to_cfar.iteritems():
    prob_to_cfar[key] = prob_to_cfar[key][0] / prob_to_cfar[key][1]
    
#prob_to_cfar

In [7]:
def convert_string_if_equation(equation):
    operator_set = set(['+', '=', '/', '*', '^', '>', '<']) # does not include minus sign.
    for char in equation:
        if char in operator_set:
            return parse_equation(equation)
    return equation
        
def parse_equation(equation):
    operator_set = set(['+', '=', '/', '*', '^', '>', '<', '@', '|', '&', 
                       ':', ';', '`', '(', ')', '#', "'", '"'])

    to_add = []
    i = 0
    equation = equation.replace('??sqrt', '@')
    equation = equation.replace('sqrt', '@')
    equation = equation.replace(' or ', '|')
    equation = equation.replace(' and ', '&')
    equation = equation.replace('log', '`')
    equation = equation.replace('abs', '#')
    equation = equation.replace('row', "'")
    equation = equation.replace('root', '"')
    while i < len(equation):

        if equation[i].isdigit():
            i += 1
            already_decimal = False
            while i < len(equation):
                if equation[i].isdigit():
                    i += 1
                elif equation[i] is ',':
                    i += 1
                elif equation[i] is '.' and not already_decimal:
                    i += 1
                    already_decimal = True
                else:
                    break

            to_add.append('~') # ~ represents number

        elif equation[i] in operator_set:
            to_add.append(equation[i])
            i += 1

        elif equation[i] is '-':

            if i == 0 or equation[i-1] in operator_set:
                i += 1
                if equation[i].isdigit():
                    i += 1
                    already_decimal = False
                    while i < len(equation):
                        if equation[i].isdigit():
                            i += 1
                        elif equation[i] is ',':
                            i += 1
                        elif equation[i] is '.' and not already_decimal:
                            i += 1
                            already_decimal = True
                        else:
                            break

                    to_add.append('~') # ~ represents number
                    #to_add.append('-~')  # this would treat all minus signs as an operator
                else:
                    to_add.append('-')

            else:
                i += 1
                to_add.append('-')

        elif equation[i].isalpha():
            i += 1
            to_add.append('x') # x is a variable name

        else:
            try:
                unicode(equation[i])
                print equation, equation[i], index
                raise AssertionError
                i += 1
            except UnicodeDecodeError:
                i += 1
                pass

    return "".join(to_add)


In [8]:
data['Step Name'] = data['Step Name'].apply(convert_string_if_equation)

In [9]:
step_to_cfar = {}

for step, cfa in zip(data['Step Name'], data['Correct First Attempt']):
    if step not in step_to_cfar:
        step_to_cfar[step] = [cfa, 1.]
    else:
        step_to_cfar[step][0] += cfa
        step_to_cfar[step][1] += 1.
        
for key, value in step_to_cfar.iteritems():
    step_to_cfar[key] = step_to_cfar[key][0] / step_to_cfar[key][1]
    
#step_to_cfar

In [11]:
subskill_to_cfar = {}
for kc, cfa in zip(data['KC(SubSkills)'], data['Correct First Attempt']):
    if kc is not np.nan:
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                if component not in subskill_to_cfar:
                    subskill_to_cfar[component] = [cfa, 1.]
                else:
                    subskill_to_cfar[component][0] += cfa
                    subskill_to_cfar[component][1] += 1.
                    
print subskill_to_cfar['Bogus skill']
                    
for key, value in subskill_to_cfar.iteritems():
    subskill_to_cfar[key] = subskill_to_cfar[key][0] / subskill_to_cfar[key][1]

subskill_to_cfar

[408861, 408861.0]


{'Bogus skill': 1.0,
 'Calculate part in proportion with fractions': 0.80298189563365285,
 'Calculate product of means or extremes': 0.89819341126461216,
 'Calculate ratio denominator from given': 0.70135383203304269,
 'Calculate ratio numerator from given': 0.68614012184508266,
 'Calculate total in proportion with fractions': 0.79958158995815898,
 'Calculate unit rate': 0.74568144095407884,
 'Changing axis bounds': 0.81431203887501513,
 'Changing axis intervals': 0.97414274198549022,
 'Choose Graphical a in A problem': 0.72402597402597402,
 'Choose Graphical a in G problem': 0.5658362989323843,
 'Choose Graphical a in N problem': 0.3632075471698113,
 'Choose Graphical a in V problem': 0.87912087912087911,
 'Choose Graphical h in A problem': 0.83720930232558144,
 'Choose Graphical h in G problem': 0.79411764705882348,
 'Choose Graphical h in N problem': 0.46666666666666667,
 'Choose Graphical h in V problem': 0.95402298850574707,
 'Choose Graphical k in A problem': 0.84805653710247353,

In [12]:
traced_to_cfar = {}
for kc, cfa in zip(data['KC(KTracedSkills)'], data['Correct First Attempt']):
    if kc is not np.nan:
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                if component not in traced_to_cfar:
                    traced_to_cfar[component] = [cfa, 1.]
                else:
                    traced_to_cfar[component][0] += cfa
                    traced_to_cfar[component][1] += 1.
                    
                    
for key, value in traced_to_cfar.iteritems():
    traced_to_cfar[key] = traced_to_cfar[key][0] / traced_to_cfar[key][1]

traced_to_cfar

{'Adding Polynomials-1': 0.86586390366844024,
 'Adding/subtracting, type-in, decimal-1': 0.61818818994217628,
 'Calculate Eliminate Parentheses (decimal)-1': 0.86432044454734891,
 'Calculate Eliminate Parentheses (distribute)-1': 0.88557993730407525,
 'Calculate Eliminate Parentheses (large, decimal)-1': 0.8446766450155323,
 'Calculate Eliminate Parentheses (numerator)-1': 0.82448979591836735,
 'Calculate Eliminate Parentheses-1': 0.90684315684315686,
 'Calculate negative coefficient-1': 0.67101226993865026,
 'Calculate part in proportion with fractions-1': 0.80298189563365285,
 'Calculate product of means or extremes-1': 0.89819341126461216,
 'Calculate ratio denominator from given-1': 0.70135383203304269,
 'Calculate ratio numerator from given-1': 0.68614012184508266,
 'Calculate solution with means and extremes using difficult numbers-1': 0.819557625145518,
 'Calculate solution with means and extremes using simple numbers-1': 0.85432016075016748,
 'Calculate total in proportion with

In [14]:
rules_to_cfar = {}
for kc, cfa in zip(data['KC(Rules)'], data['Correct First Attempt']):
    if kc is not np.nan:
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                if component not in rules_to_cfar:
                    rules_to_cfar[component] = [cfa, 1.]
                else:
                    rules_to_cfar[component][0] += cfa
                    rules_to_cfar[component][1] += 1.
                    
                    
for key, value in rules_to_cfar.iteritems():
    rules_to_cfar[key] = rules_to_cfar[key][0] / rules_to_cfar[key][1]

rules_to_cfar

{'UNIT-CONVERSION-CONVERTED-VALUE-DEFAULT-HINTS': 0.86576086956521736,
 '[Rule: CLT expression, any action (null,right)]': 0.0,
 'RATIONAL-PLOT-PERCENT-GTE-HUNDRED-RATIONAL-ON-MINOR-TICK-HINTS': 0.56551297898640296,
 'SET-MAX-BOUND-BOUND-LOW': 0.30129516019086572,
 'GRAPHER-POINT-GOOD-FIX-Y-MAX': 0.071428571428571425,
 'null, Region is a segment, Absolute value of expression problem': 0.96129486277269527,
 'RATIO-LABEL-GOAL-NODE-ConditionalHint43264': 0.92308506360192177,
 'generic-label-hint': 0.33333333333333331,
 'RATIONAL-ORDERINGS-BUG-LATER-ORDERING-NUMBERLINE-AND-SMALLER-DROPDOWNS-FINISHED': 0.93139510562643801,
 'CALCULATE-OTHER-DEPENDENT-VARIABLE': 0.94568087514145605,
 'PM-ROW-2-SINGLE': 0.9720338983050848,
 '[Rule: general RF noMult catchall ([SolverOperation rf],)]': 0.61068702290076338,
 'RATIO-GOAL-NODE-ConditionalHint43277': 0.9458159509202454,
 'null, Problem type V, Hints for H parameter, GRAPHIC-V-NEG-H, GRAPHIC-V-NEG-H-FIRST': 0.96052631578947367,
 'null, Region is a 

### Build the columns for training set

In [15]:
print 'Building id-CFAR column'
id_cfar = [id_to_cfar[stu_id] for stu_id in data['Anon Student Id']]


Building id-CFAR column


In [17]:
print 'Building unit-CFAR column'
unit_cfar = [unit_to_cfar[unit] for unit in data['Unit']]

Building unit-CFAR column


In [19]:
print 'Building section-CFAR column'
section_cfar = [section_to_cfar[section] for section in data['Sections']]

Building section-CFAR column


In [21]:
print 'Building prob-CFAR column'
prob_cfar = [prob_to_cfar[prob] for prob in data['Problem Name']]

Building prob-CFAR column


In [23]:
print 'Building step-CFAR column'
step_cfar = [step_to_cfar[step] for step in data['Step Name']]

Building step-CFAR column


In [26]:
cfa_mean = data['Correct First Attempt'].mean()
cfa_mean

0.85950115041160213

In [28]:
print 'Building subskill-CFAR column'
subskill_cfar = []
for kc in data['KC(SubSkills)']:
    if kc is np.nan:
        subskill_cfar.append(cfa_mean)
    else:
        to_average = []
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                to_average.append(subskill_to_cfar[component])
        subskill_cfar.append(np.mean(to_average))
        
        

Building subskill-CFAR column


In [31]:
print 'Building traced-CFAR column'
traced_cfar = []
for kc in data['KC(KTracedSkills)']:
    if kc is np.nan:
        traced_cfar.append(cfa_mean)
    else:
        to_average = []
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                to_average.append(traced_to_cfar[component])
        traced_cfar.append(np.mean(to_average))
        
        

Building traced-CFAR column


In [33]:
print 'Building rules-CFAR column'
rules_cfar = []
for kc in data['KC(Rules)']:
    if kc is np.nan:
        rules_cfar.append(cfa_mean)
    else:
        to_average = []
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                to_average.append(rules_to_cfar[component])
        rules_cfar.append(np.mean(to_average))
        
        

Building rules-CFAR column


In [34]:
zip(subskill_cfar[:10], traced_cfar[:10], rules_cfar[:10])

[(0.83501812034287359, 0.85950115041160213, 0.88112466704891024),
 (0.94156477980145614, 0.85950115041160213, 0.95109688148395666),
 (0.61189159874434773, 0.65013147069470278, 0.53869704988300127),
 (0.82756253141652414, 0.85112866510071372, 0.81938342120679553),
 (0.74967294144515073, 0.7021989388772597, 0.8216176289548196),
 (0.86504316389894653, 0.85112866510071372, 0.89916358474905089),
 (0.74967294144515073, 0.7021989388772597, 0.70453738091393514),
 (0.84429382280908938, 0.85112866510071372, 0.89916358474905089),
 (0.74967294144515073, 0.7021989388772597, 0.70453738091393514),
 (0.85950115041160213, 0.85950115041160213, 0.78742939377585897)]

In [36]:
opp_mean = []
for opp1, opp2, opp3 in zip(data['Opportunity(SubSkills)'], 
                            data['Opportunity(KTracedSkills)'], 
                            data['Opportunity(Rules)']):
    to_average = []
    if opp1 is not np.nan:
        for opp in opp1.split('~~'):
            to_average.append(float(opp))
    if opp2 is not np.nan:
        for opp in opp2.split('~~'):
            to_average.append(float(opp))
    if opp3 is not np.nan:
        for opp in opp3.split('~~'):
            to_average.append(float(opp))
    
    if to_average:
        opp_mean.append(np.mean(to_average))

print opp_mean[:100]
opp_mean = np.mean(opp_mean)

[1.0, 1.0, 1.0, 1.0, 1.5, 1.5, 2.375, 2.5, 3.375, 1.0, 1.0, 2.0, 3.0, 2.0, 3.5, 4.5, 3.1000000000000001, 4.875, 5.875, 2.0, 2.0, 4.0, 5.0, 3.0, 4.9000000000000004, 4.75, 7.375, 4.5999999999999996, 7.875, 7.0, 9.25, 3.0, 3.0, 6.0, 7.0, 4.0, 7.0, 5.4000000000000004, 10.75, 7.25, 11.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.3333333333333333, 1.5, 2.0, 1.5714285714285714, 2.3333333333333335, 3.0, 3.0, 2.3333333333333335, 3.0, 3.5, 2.1428571428571428, 3.6666666666666665, 4.666666666666667, 3.3333333333333335, 4.5, 5.0, 2.8571428571428572, 4.0, 5.666666666666667, 5.333333333333333, 4.333333333333333, 6.0, 6.5, 3.5714285714285716, 5.333333333333333, 7.333333333333333, 5.333333333333333, 7.5, 8.0, 4.2857142857142856, 6.333333333333333, 8.3333333333333339, 3.6666666666666665, 9.0, 9.5, 4.2857142857142856, 7.333333333333333, 9.3333333333333339, 6.666666666666667, 10.5, 11.0, 5.7142857142857144, 8.3333333333333339, 10.333333333333334, 5.0, 12.0, 12.5, 5.8571428571428568, 7.666666666666667, 11

In [38]:
print 'Building opportunity-column'
opportunity_column = []


for subskill, subskill_opp, traced, traced_opp, rules, rules_opp in zip(data['KC(SubSkills)'], 
                                                                        data['Opportunity(SubSkills)'], 
                                                                        data['KC(KTracedSkills)'], 
                                                                        data['Opportunity(KTracedSkills)'], 
                                                                        data['KC(Rules)'], 
                                                                        data['Opportunity(Rules)']):
    opportunities = []
    
    if subskill is not np.nan:
        component_set = set([])
        for component, cnt in zip(subskill.split('~~'), subskill_opp.split('~~')):
            if component not in component_set:
                component_set.add(component)
                opportunities.append(float(cnt))
    
    if traced is not np.nan:
        component_set = set([])
        for component, cnt in zip(traced.split('~~'), traced_opp.split('~~')):
            if component not in component_set:
                component_set.add(component)
                opportunities.append(float(cnt))
    
    if rules is not np.nan:
        component_set = set([])
        for component, cnt in zip(rules.split('~~'), rules_opp.split('~~')):
            if component not in component_set:
                component_set.add(component)
                opportunities.append(float(cnt))
    
    
    if not opportunities:
        opportunity_column.append(opp_mean)
    else:
        opportunity_column.append(np.mean(opportunities))
        
        

Building opportunity-column


In [41]:
basic_condensed_features = np.array([id_cfar, unit_cfar, section_cfar, prob_cfar, 
                                  step_cfar, subskill_cfar, traced_cfar, 
                                  rules_cfar, opportunity_column, data['Problem View'].tolist()]).T

In [43]:
basic_condensed_features.shape

(8483920, 10)

In [54]:
joblib.dump(basic_condensed_features, 'algebra_2005_2008_pickles/basic_condensed_features_matrix_(no_temporal).compressed', compress=3)

['algebra_2005_2008_pickles/basic_condensed_features_matrix_(no_temporal).compressed']

## Build the columns for testing set

In [45]:
testdata = pd.read_csv('algebra_2008_2009_test.txt', delimiter='\t')
split_object = testdata['Problem Hierarchy'].str.split(', ')
testdata['Unit'] = split_object.str[0]
testdata['Sections'] = split_object.str[1]
del testdata['Problem Hierarchy']# if you choose to use it, convert step names
testdata['Step Name'] = testdata['Step Name'].apply(convert_string_if_equation)

In [52]:
print 'Building id-CFAR column'
t_id_cfar = [id_to_cfar[stu_id] for stu_id in testdata['Anon Student Id']]
len(t_id_cfar)

Building id-CFAR column


508912

In [53]:
t_unit_cfar = [unit_to_cfar[unit] for unit in testdata['Unit']]
len(t_unit_cfar)

508912

In [55]:
t_section_cfar = [section_to_cfar[section] for section in testdata['Sections']]
len(t_section_cfar)

508912

In [58]:
t_prob_cfar = []
for prob in testdata['Problem Name']:
    if prob in prob_to_cfar:
        t_prob_cfar.append(prob_to_cfar[prob])
    else:
        t_prob_cfar.append(cfa_mean)
len(t_prob_cfar)

508912

In [59]:
t_step_cfar = []
for step in testdata['Step Name']:
    if step in step_to_cfar:
        t_step_cfar.append(step_to_cfar[step])
    else:
        t_step_cfar.append(cfa_mean)
len(t_step_cfar)

508912

In [60]:
print 'Building subskill-CFAR column'
t_subskill_cfar = []
for kc in testdata['KC(SubSkills)']:
    if kc is np.nan:
        t_subskill_cfar.append(cfa_mean)
    else:
        to_average = []
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                to_average.append(subskill_to_cfar[component])
        t_subskill_cfar.append(np.mean(to_average))
        
len(t_subskill_cfar)

Building subskill-CFAR column


508912

In [61]:
print 'Building subskill-CFAR column'
t_traced_cfar = []
for kc in testdata['KC(KTracedSkills)']:
    if kc is np.nan:
        t_traced_cfar.append(cfa_mean)
    else:
        to_average = []
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                to_average.append(traced_to_cfar[component])
        t_traced_cfar.append(np.mean(to_average))
        
len(t_traced_cfar)

Building subskill-CFAR column


508912

In [63]:
print 'Building subskill-CFAR column'
t_rules_cfar = []
for kc in testdata['KC(Rules)']:
    if kc is np.nan:
        t_rules_cfar.append(cfa_mean)
    else:
        to_average = []
        component_set = set([])
        for component in kc.split('~~'):
            if component not in component_set:
                component_set.add(component)
                if component not in rules_to_cfar:
                    to_average.append(cfa_mean)
                else:
                    to_average.append(rules_to_cfar[component])
        t_rules_cfar.append(np.mean(to_average))
        
len(t_rules_cfar)

Building subskill-CFAR column


508912

In [64]:
zip(t_subskill_cfar, t_traced_cfar, t_rules_cfar)[:10]

[(0.85950115041160213, 0.85950115041160213, 0.78742939377585897),
 (0.85950115041160213, 0.85950115041160213, 0.80353920192500505),
 (0.85950115041160213, 0.85950115041160213, 0.88112466704891024),
 (0.83501812034287359, 0.85950115041160213, 0.88112466704891024),
 (0.94156477980145614, 0.85950115041160213, 0.95109688148395666),
 (0.6116297274631779, 0.68374915153460658, 0.63977638096341571),
 (0.86504316389894653, 0.85112866510071372, 0.87973581423744807),
 (0.78029626013146169, 0.73748404668983847, 0.83370340696654077),
 (0.84429382280908938, 0.85112866510071372, 0.87973581423744807),
 (0.78029626013146169, 0.73748404668983847, 0.83370340696654077)]

In [72]:
print 'Building opportunity-column'
t_opportunity_column = []


for subskill, subskill_opp, traced, traced_opp, rules, rules_opp in zip(testdata['KC(SubSkills)'], 
                                                                        testdata['Opportunity(SubSkills)'], 
                                                                        testdata['KC(KTracedSkills)'], 
                                                                        testdata['Opportunity(KTracedSkills)'], 
                                                                        testdata['KC(Rules)'], 
                                                                        testdata['Opportunity(Rules)']):
    opportunities = []
    
    if subskill is not np.nan:
        component_set = set([])
        for component, cnt in zip(subskill.split('~~'), subskill_opp.split('~~')):
            if component not in component_set:
                component_set.add(component)
                opportunities.append(float(cnt))
    
    if traced is not np.nan:
        component_set = set([])
        for component, cnt in zip(traced.split('~~'), traced_opp.split('~~')):
            if component not in component_set:
                component_set.add(component)
                opportunities.append(float(cnt))
    
    if rules is not np.nan:
        component_set = set([])
        for component, cnt in zip(rules.split('~~'), rules_opp.split('~~')):
            if component not in component_set:
                component_set.add(component)
                opportunities.append(float(cnt))
    
    
    if not opportunities:
        t_opportunity_column.append(opp_mean)
    else:
        t_opportunity_column.append(np.mean(opportunities))
        
        

Building opportunity-column


In [75]:
len(t_opportunity_column)

508912

In [76]:
t_basic_condensed_features = np.array([t_id_cfar, t_unit_cfar, t_section_cfar, t_prob_cfar, 
                                  t_step_cfar, t_subskill_cfar, t_traced_cfar, 
                                  t_rules_cfar, t_opportunity_column, testdata['Problem View'].tolist()]).T

In [78]:
t_basic_condensed_features.shape

(508912, 10)

In [79]:
joblib.dump(t_basic_condensed_features, 'algebra_2005_2008_pickles/basic_test_condensed_features_matrix_(no_temporal)', compress=3)

['algebra_2005_2008_pickles/basic_test_condensed_features_matrix_(no_temporal)']